### Solace Relativity v1.0 - Build Relativity and EJ Weighting

Read exploits db, use derivedValueIndicators and build data-values files and relativity scores, build expert-judgement template files, and build expert-judgement weighting files.

In [ ]:
import pandas as pd
import json
import numpy as np
import math
import os

#WIP

## Load Exploits, Protocols(tbd), and mappingTables (transactional and temporal) 
temporalDb=pd.read_excel('../temporalDB/TemporalDataset.xlsx')
exploitsDf = pd.read_csv('../exploits/historical_exploits.csv')
f = open('../governance/reference/mappingTablesSolace.json')
lookupTables = json.loads(f.read())
f.close()

# Init dfs for lookup tables
tvlSolaceLookupTable = pd.DataFrame(lookupTables['tvlSolaceLookup'])
auditsSolaceLookupTable = pd.DataFrame(lookupTables['auditSolaceLookup'])
oraclesSolaceLookupTable = lookupTables['oracleSolaceLookup'] # just values, not mappings: yes/no/unknown
categorySolaceLookupTable=pd.DataFrame(lookupTables['categorySolaceLookup'])



# Calc Exploit Damage ratio
exploitsDf['tvlPreExploit'] = exploitsDf['tvlPreExploit'] / 10 # confirm wtih Tim
exploitsDf['exploitDamage'] = exploitsDf['loss'] / exploitsDf['tvlPreExploit']
#exploitsDf['exploitDamage']= exploitsDf['exploitDamage'].astype(object).replace(np.nan, 'null')

# Calc time to exploit
exploitsDf[['launchDate','exploitDate']] = exploitsDf[['launchDate','exploitDate']].apply(pd.to_datetime) #if conversion required
exploitsDf['timeToExploit'] = (exploitsDf['exploitDate'] - exploitsDf['launchDate']).dt.days
exploitsDf['timeToExploit'] = exploitsDf['timeToExploit'].astype(object).replace(np.nan, 'null')


# Derive tvlSolace column
def lookup_tvlSolace(tvl):
    if math.isnan(tvl):
        return 'null'
    match = (tvlSolaceLookupTable['lowerLimit'] <= tvl) & (tvlSolaceLookupTable['upperLimit'] > tvl)
    tvlOut = tvlSolaceLookupTable['tvlSolace'][match]
    return tvlOut.values[0]


# Derive auditSolace column
def lookup_auditSolace(audits):
    if math.isnan(audits):
        return 'Unknown'
    match = (auditsSolaceLookupTable['auditsOg'] == audits)
    auditsOut = auditsSolaceLookupTable['auditSolace'][match]
    return auditsOut.values[0]


# Derive oracleSolace column
def lookup_oracleSolace(oracles,catalog):
    if pd.isna(catalog):
        return 'Unknown'
    elif pd.isna(oracles):
        return 'No'
    else:
        return "Yes"

# Reminder-add categorySolace to exploits too

#Solace judgement

exploitsDf['tvlSolace'] = exploitsDf['tvlPreExploit'].apply(lookup_tvlSolace)
temporalDb['tvlSolace'] = temporalDb['tvl'].apply(lookup_tvlSolace)
exploitsDf['auditSolace'] = exploitsDf['audits'].apply(lookup_auditSolace)
temporalDb['auditSolace'] = temporalDb['audits'].apply(lookup_auditSolace)            
exploitsDf['oracleSolace'] = exploitsDf.apply(lambda x: lookup_oracleSolace(x.oracles, x.category), axis=1)
temporalDb['oracleSolace'] = temporalDb.apply(lambda x: lookup_oracleSolace(x.oracles, x.tags), axis=1)


# Create columns for use in one way calcs
exploitsDf['tvlAvgDamage'] = exploitsDf.groupby(['tvlSolace'])['exploitDamage'].transform('mean')
exploitsDf['auditAvgDamage'] = exploitsDf.groupby(['auditSolace'])['exploitDamage'].transform('mean')
exploitsDf['oracleAvgDamage'] = exploitsDf.groupby(['oracleSolace'])['exploitDamage'].transform('mean')

averageDamage = exploitsDf['exploitDamage'].mean()

exploitsDf['tvlDamageRatio'] = exploitsDf['tvlAvgDamage'] / averageDamage
exploitsDf['auditDamageRatio'] = exploitsDf['auditAvgDamage'] / averageDamage
exploitsDf['oracleDamageRatio'] = exploitsDf['oracleAvgDamage'] / averageDamage

print('Average exploit damage ratio: ',averageDamage)

# Create one way 'tables', separate for readability 
# TVL
tvlOneWay = pd.DataFrame()
tvlOneWay['damageRatio'] = exploitsDf['tvlDamageRatio'].unique()
tvlOneWay['tvlSolace'] =exploitsDf['tvlSolace'].unique()
tvlOneWay['avgDamage'] = exploitsDf['tvlAvgDamage'].unique()

# AUDITS
auditOneWay = pd.DataFrame()
auditOneWay['damageRatio'] = exploitsDf['auditDamageRatio'].unique()
auditOneWay['auditSolace'] = exploitsDf['auditSolace'].unique()
auditOneWay['avgDamage'] = exploitsDf['auditAvgDamage'].unique()

# ORACLES
oracleOneWay = pd.DataFrame()
oracleOneWay['damageRatio'] = exploitsDf['oracleDamageRatio'].unique()
oracleOneWay['oracleSolace'] =exploitsDf['oracleSolace'].unique()
oracleOneWay['avgDamage'] = exploitsDf['oracleAvgDamage'].unique()

def write_json(df, fileName, drop):
    if drop == True:
        df.drop(['avgDamage'], axis=1, inplace=True)
    data = df.to_json(orient='columns')
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    with open(fileName, 'w') as outfile:
        outfile.write(data)

write_json(pd.merge(tvlOneWay, tvlSolaceLookupTable, how="right", on='tvlSolace'), '../governance/data-values/tvlDataValues.json', True)
write_json(pd.merge(auditOneWay, auditsSolaceLookupTable, how="right", on='auditSolace'), '../governance/data-values/auditDataValues.json', True)
write_json(oracleOneWay, '../governance/data-values/oracleDataValues.json', True)

print("Data values json files created in governance directory")

# Initialize files and weightings

# EJ weighting files
def create_weights(predictor):
    weightingsTemplate = '{"' + predictor + 'Basis": {"1": "data","2":"Ej1"},"weight": {"1": 0.25, "2":0.75}}'
    weightingsTemplateJson = json.loads(weightingsTemplate)
    weightingsTemplateDf = pd.DataFrame(weightingsTemplateJson)
    write_json(weightingsTemplateDf, '../governance/weightings/'+predictor+'Weighting.json', False)
    
create_weights('audit')
create_weights("oracle")
create_weights("tvl")
create_weights("category")




### Temporal Data relativities

In [32]:
# Data values
def open_data(file):
    Data = open(file)
    lookupTables = json.loads(Data.read())
    Data.close()
    return lookupTables

def add_column(file,key,database,merger):
    lookup=open_data(file)
    attributeSolaceRelativity=lookup[key]
    attributeFrame=pd.DataFrame(attributeSolaceRelativity)
    combinedAuditTable1=pd.merge(database,attributeFrame,left_on=merger, right_on=merger, how='left')
    return combinedAuditTable1

def calc_finalRelativity(weightLookup,combinedtable2,column1,column2,key,key_weight):
    lookupTable=open_data(weightLookup)
    lookupFrame=pd.DataFrame(lookupTable)
    data_weight=lookupFrame[lookupFrame[key_weight]=='data']['weight']
    solaceRM_weight=lookupFrame[lookupFrame[key_weight]=='Ej1']['weight']
    combinedtable2[key]=(column1*data_weight.values)+(column2*solaceRM_weight.values) #+(column1*solaceRM_weight.values)+(column2*solaceRM_weight.values)
    return combinedtable2

def aggregateTables(weightLookup,file,file2,key,key1,key2,database,merger,key_weight):
    column_test=add_column(file,key,database,merger)
    column_test2=add_column(file2,key,column_test,merger)
    column_test3=calc_finalRelativity(weightLookup,column_test2,column_test2[key1],column_test2[key2],key,key_weight)
    return column_test3

AuditRelativityColumns=aggregateTables('../governance/weightings/auditWeighting.json','../governance/data-values/auditRelativities.json','../governance/expert-judgement/solaceRM/ejAuditDataValues.json','auditSolaceRelativities','relativityAuditData','relativityAuditEj1',temporalDb,'auditSolace','auditBasis')
TVLRelativityColumns=aggregateTables('../governance/weightings/tvlWeighting.json','../governance/data-values/tvlRelativities.json','../governance/expert-judgement/solaceRM/ejTvlDataValues.json','tvlSolaceRelativities','relativityTvlData','relativityTvlEj1',AuditRelativityColumns,'tvlSolace','tvlBasis')
OracleRelativityColumns=aggregateTables('../governance/weightings/oracleWeighting.json','../governance/data-values/oracleRelativities.json','../governance/expert-judgement/solaceRM/ejOracleDataValues.json','oracleSolaceRelativities','relativityOracleData','relativityOracleEj1',TVLRelativityColumns,'oracleSolace','oracleBasis')
CategoryRelativityColumns=aggregateTables('../governance/weightings/categoryWeighting.json','../governance/data-values/categoryRelativities.json','../governance/expert-judgement/solaceRM/ejCategoryDataValues.json','categorySolaceRelativities','relativityCategoryData','relativityCategoryEj1',OracleRelativityColumns,'categorySolace','categoryBasis')

CategoryRelativityColumns['finalRelativity']=CategoryRelativityColumns['auditSolaceRelativities']*CategoryRelativityColumns['tvlSolaceRelativities']*CategoryRelativityColumns['oracleSolaceRelativities']*CategoryRelativityColumns['categorySolaceRelativities']


CategoryRelativityColumns


,name,address,symbol,audits,chains,oracles,tvl,forkedFrom,SolaceID,appId,...,relativityTvlData,relativityTvlEj1,tvlSolaceRelativities,relativityOracleData,relativityOracleEj1,oracleSolaceRelativities,relativityCategoryData,relativityCategoryEj1,categorySolaceRelativities,finalRelativity
0,Curve,0xd533a949740bb3306d119cc777fa900ba034cd52,CRV,2.0,"['Ethereum', 'Avalanche', 'Fantom', 'Polygon',...",['Chainlink'],2.003971e+10,NaN,curve,curve,...,1,1,1.0,1,1,1.0,1,1,1.0,1.0
1,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2.0,"['Ethereum', 'Avalanche', 'Polygon']",['Chainlink'],1.409376e+10,NaN,aave,aave-amm,...,1,1,1.0,1,1,1.0,1,1,1.0,1.0
2,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2.0,"['Ethereum', 'Avalanche', 'Polygon']",['Chainlink'],1.409376e+10,NaN,aave,aave-safety-module,...,1,1,1.0,1,1,1.0,1,1,1.0,1.0
3,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2.0,"['Ethereum', 'Avalanche', 'Polygon']",['Chainlink'],1.409376e+10,NaN,aave,aave,...,1,1,1.0,1,1,1.0,1,1,1.0,1.0
4,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2.0,"['Ethereum', 'Avalanche', 'Polygon']",['Chainlink'],1.409376e+10,NaN,aave,aave-v2,...,1,1,1.0,1,1,1.0,1,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,ARCx,eth:0x1321f1f1aa541a56c31682c57b80ecfccd9bb288,ARCx,0.0,['Ethereum'],NaN,0.000000e+00,NaN,arcx,arcx,...,1,3,2.5,1,1,1.0,1,1,1.0,2.5
129,The Sandbox,0x3845badade8e6dff049820680d1f14bd3903a5d0,SAND,0.0,"['Ethereum', 'Polygon']",NaN,0.000000e+00,NaN,the,the-graph,...,1,3,2.5,1,1,1.0,1,1,1.0,2.5
130,LooksRare,0xf4d2888d29d722226fafa5d9b24f9164c092421e,LOOKS,0.0,['Ethereum'],NaN,0.000000e+00,NaN,looksrare,looksrare,...,1,3,2.5,1,1,1.0,1,1,1.0,2.5
131,The Open DAO SOS,0x3b484b82567a09e2588a13d54d032153f0c0aee0,SOS,2.0,['Ethereum'],NaN,0.000000e+00,NaN,the,the-graph,...,1,3,2.5,1,1,1.0,1,1,1.0,2.5


### Map protocols to tiers based on final Relativity

In [33]:
RelativityTable=pd.read_json('../governance/reference/relativityTable.json')


def tiers(relativities,RelativityTable):
    min_val=RelativityTable['minRelativity']
    max_val=RelativityTable['maxRelativity']
    tier=RelativityTable['tier']
    riskTiers=[]
    for value in relativities:
        if value<=max_val[0] and value>=min_val[0]:
                riskTiers.append(tier[0])
        elif value<=max_val[1] and value>=min_val[1]:
                riskTiers.append(tier[1])
        elif value<=max_val[2] and value>=min_val[2]:
                riskTiers.append(tier[2])
        else:
                riskTiers.append(tier[3])
    return riskTiers



protocolRelativity=CategoryRelativityColumns['finalRelativity']
Risk_Tiers=tiers(protocolRelativity,RelativityTable)
CategoryRelativityColumns['tier']=Risk_Tiers

protocolMap=CategoryRelativityColumns.filter(['appId','tier','categorySolace'], axis=1)
protocolMap=protocolMap.rename(columns={'categorySolace':'category'})


with pd.ExcelWriter('../governance/reference/SolaceRMValues.xlsx', mode="a",engine="openpyxl",if_sheet_exists="replace") as writer:protocolMap.to_excel(writer, sheet_name="protocolMap")

